In [6]:
from transformers import LlamaForCausalLM
model_hf = LlamaForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
print(model_hf)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [7]:
import torch

In [29]:
'''
input: (batch, sequence_length, embedding_size)
algorithm:
    fomula: RMS(x) = sqrt(mean(x_i^2) + epsilon)
    output: norm_x = (x / RMS(x)) * gamma
'''
gamma = torch.tensor([2, 2, 2])
eps = 1e-6
x = torch.tensor([[[1, 2, 3], [4, 5, 6]]], dtype=torch.float32)
print(x.shape)
RMS = torch.sqrt(torch.mean(x**2, dim=-1, keepdim=True) + eps)
print(RMS.shape)
out = x / RMS * gamma
print(out)

torch.Size([1, 2, 3])
torch.Size([1, 2, 1])
tensor([[[0.9258, 1.8516, 2.7775],
         [1.5791, 1.9739, 2.3686]]])
